In [237]:
import numpy as np
import matplotlib.pyplot as plt

In [243]:
# Global parameters
N = 100  # no. data points
N_iters = 10000  # max no. iterations before stopping
noise_std = 0.1
beta_tol = 1e-10  # beta tolerance for stopping iteration when |beta_new - beta_old| <= beta_tol
lmbda = 0.01

# Create data set
np.random.seed(2023)
x = np.random.rand(N, 1)
noise = np.random.normal(0, noise_std, x.shape)
y = 5 - 10*x + 2*x**2 + noise

# Analytical calculations for comparing
X = np.c_[np.ones((N, 1)), x]  # ??
beta_ols = np.linalg.inv(X.T @ X) @ (X.T @ y)  # own OLS inversion
I = np.identity(np.shape(X.T @ X)[0])
beta_ridge = np.linalg.inv(X.T @ X + lmbda*I) @ (X.T @ y)  # own Ridge inversion

H = 2/N * X.T @ X  # Hessian matrix
eig_vals, eig_vecs = np.linalg.eig(H)

# Plain gradient descent (GD)

In [239]:
# GD Parameters
learn_rate = 0.4

# Iterate through and improve beta
beta = np.random.randn(2, 1)
beta_prev = 0
i = 0
while i < N_iters and all(abs(beta - beta_prev) > beta_tol):  # convergence test
    beta_prev = beta.copy()
    grad = 2/N * X.T @ (X @ beta - y)
    beta -= learn_rate * grad
    i += 1
iters_gd = i
beta_gd = beta

# Momentum based GD

In [240]:
# Momentum GD Parameters
learn_rate = 0.4
momentum_rate = 0.642

# Iterate through and improve beta
beta_pp = np.random.randn(2, 1)
beta_p = 2/N * X.T @ (X @ beta_pp - y)
beta = 2/N * X.T @ (X @ beta_p - y)
i = 0
while i < N_iters - 2 and all(abs(beta - beta_p) > beta_tol):  # convergence test
    beta_pp = beta_p.copy()
    beta_p = beta.copy()
    grad = 2/N * X.T @ (X @ beta - y)
    beta += momentum_rate * (beta_p - beta_pp) - learn_rate * grad
    i += 1
iters_mgd_ols = i
beta_mgd_ols = beta

# Stochastic gradient descent (SDG)

In [241]:
# SDG parameters
learn_rate = 0.4
M = N/100  # size of each minibatch
m = int(N/M)  # number of minibatches
N_epochs = 50  # number of iterations over all minibatches 



# Print results

In [242]:
print(f"Own inversion:\n"
      f"1/max(eigenvalues)={1/max(eig_vals):g}\n"
      f"OLS beta={beta_ols.ravel()}\n"
      f"Ridge beta={beta_ridge.ravel()}\n"
      f"Beta tolerance={beta_tol}\n"
      f"Max iterations={N_iters}\n")
print(f"Own GD code:\n"
      f"Learning rate={learn_rate}\n"
      f"OLS beta={beta_gd_ols.ravel()}\n"
      f"OLS iterations={iters_gd_ols}\n"
      f"Ridge beta={beta_gd_ridge.ravel()}\n"
      f"Ridge iterations={iters_gd_ridge}\n")
print(f"Own GD w/momentum code:\n"
      f"Learning rate={learn_rate}\n"
      f"momentum rate={momentum_rate}\n"
      f"OLS beta={beta_mgd_ols.ravel()}\n"
      f"OLS iterations={iters_mgd_ols}\n"
      f"Ridge beta={beta_mgd_ridge.ravel()}\n"
      f"Ridge iterations={iters_mgd_ridge}\n")

Own inversion:
1/max(eigenvalues)=0.401248
OLS beta=[ 4.65168156 -8.00899928]
Ridge beta=[ 4.64446663 -7.99497541]
beta tolerance=1e-10
Max iterations=10000

Own GD code:
Learning rate=0.4
beta=[ 4.65168156 -8.00899928]
Iterations=445

Own GD w/momentum code:
Learning rate=0.4
momentum rate=0.642
beta=[ 4.65168156 -8.00899928]
Iterations=93
